In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


If you set inplace = True , the drop() method will delete rows or columns directly from the original dataframe
df.dropna(axis='index', how='all', inplace=True)


first check if any features have missing values, -- fill missing values with mean (number) or mode (category) values of that column 

In [ ]:
if os.path.exists("/kaggle/working/submission.csv"):
  os.remove("/kaggle/working/submission.csv")
if os.path.exists("/kaggle/temp/train.csv"):
  os.remove("/kaggle/temp/train.csv")
if os.path.exists("/kaggle/temp/test.csv"):
  os.remove("/kaggle/temp/test.csv")
if os.path.exists("/kaggle/temp/ecfp.csv"):
  os.remove("/kaggle/temp/ecfp.csv")
if os.path.exists("/kaggle/temp/cb_embeddings.csv"):
  os.remove("/kaggle/temp/cb_embeddings.csv")

In [ ]:
%pwd

In [3]:
%mkdir /kaggle/temp

mkdir: cannot create directory '/kaggle/temp': File exists


In [ ]:
%cd /kaggle/temp

In [ ]:
%ls
for dirname, _, filenames in os.walk('/kaggle/temp'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
!pip install duckdb
!pip install rdkit

In [ ]:
# from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import duckdb
import gc
from rdkit import Chem
from rdkit.Chem import AllChem
#from rdkit.Chem import Draw, Descriptors
#import mapply
# import matplotlib.pyplot as plt
# import seaborn as sns
RANDOM_STATE = 2509
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
#from xgboost import XGBClassifier
from catboost import CatBoostClassifier
#from lightgbm import LGBMClassifier
from sklearn.metrics import average_precision_score

**PART 1: TRAINING**
The train data only has triazine cores, but the test data has other unique cores. Building validation sets where we identify how well we are doing at scaffold hopping may be a bit tricky, since the train set only has triazine cores for us to work with.

In [ ]:
# train_path = '/kaggle/input/leash-BELKA/train.parquet'
# con = duckdb.connect()
# # train = con.query(f"""(SELECT COUNT(binds)  FROM parquet_scan('{train_path}'))""").df()
# #train.head()
# train = con.query(f"""(SELECT COUNT(binds)  FROM parquet_scan('{train_path}') WHERE binds > 0.8 )""").df()
# #train = con.query(f"""(SELECT DISTINCT COUNT(binds)  FROM parquet_scan('{train_path}') WHERE binds > 0.75 )""").df()
# con.close()
# train.head()

In [ ]:
# del con, train
# gc.collect()

In [ ]:
# con = duckdb.connect()
# train = con.query(f"""(SELECT * FROM parquet_scan('{train_path}') LIMIT 1)""").df()
# con.close()
# train.info()

In [ ]:
# del con, train
# gc.collect()

In [ ]:
extractby2=1500000
###extractby2=5000
extract=2*extractby2

train_path = '/kaggle/input/leash-BELKA/train.parquet'
con = duckdb.connect()

con.query(f"""(SELECT molecule_smiles,protein_name,binds 
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT {extractby2})
                        UNION ALL
                        (SELECT molecule_smiles,protein_name,binds
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT {extractby2})""").df().to_csv('/kaggle/temp/train.csv', index=False)

con.close()
del con
gc.collect()

In [ ]:
# ecfp_file='/kaggle/temp/ecfp.csv'

# def generate_ecfp(molecule, radius=3, bits=1024):
#     if molecule is None:
#         return None
#     return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

In [ ]:
# train = pd.read_csv('/kaggle/temp/train.csv', usecols=['molecule_smiles']) 
# train.head()

In [ ]:
# num=300000
# for i in range(num,3000000,num):  
#     selected_rows = train.iloc[i-num:i] 
#     selected_rows['molecule']= selected_rows['molecule_smiles'].apply(Chem.MolFromSmiles)
#     selected_rows=selected_rows.drop(columns=['molecule_smiles'])
#     selected_rows['ecfp'] = selected_rows['molecule'].apply(generate_ecfp)
#     selected_rows=selected_rows.drop(columns=['molecule'])
#     selected_rows['ecfp'].to_csv(ecfp_file, index=False, mode='a',header=not os.path.exists(ecfp_file))
#     del selected_rows
#     gc.collect()

    
# del train
# gc.collect()

In [ ]:
# hi=pd.read_csv('/kaggle/temp/ecfp.csv',nrows=5)
# hi.head()

**Molecular fingerprints** are numerical representations that encode molecular structures, usually as a binary vector. Each element (called a bit) of a fingerprint vector represents the presence of specific atomic or structural features within the chemical compound.

Morgan Fingerprints: a fixed-length bit fingerprint based on hashed topological features of atoms and their bond connectivities within a specified radius. Also referred to as Extended-Connectivity Fingerprints (ECFPs)
Note that different values can be specified for radius and nBits when generating Morgan Fingerprints.
radius: number of bond steps to include from each atom when calculating the fingerprint. A radius of 1 means that only the immediate neighbors are considered; a radius of 2 includes neighbors up to two bonds away, etc.
nBits: total number of bits in the fingerprint (i.e. resolution). Larger nBits values increases feature dimensionality and typically increases feature sparsity.
Bit Collision refers to the scenario where different substructural features are mapped to the same bit position. This is undesirable as it reduces the fingerprint's ability to differentiate molecules. Increasing nBits increases the available feature space and thus lowers the chance of bit collisions.

In [ ]:
# # One-hot encode the protein_name
# train = pd.read_csv('/kaggle/temp/train.csv', usecols=['protein_name']) 

# onehot_encoder = OneHotEncoder(sparse_output=False)
# protein_onehot = onehot_encoder.fit_transform(train['protein_name'].values.reshape(-1, 1))
# del train 
# gc.collect()

In [ ]:
# train=pd.read_csv('/kaggle/temp/ecfp.csv',converters={'ecfp': pd.eval})
# X_train = [ecfp + protein for ecfp, protein in zip(train['ecfp'].apply(eval).apply(np.array).tolist(), protein_onehot.tolist())]
# del train, protein_onehot, onehot_encoder
# gc.collect()

In [ ]:
# train = pd.read_csv('/kaggle/temp/train.csv', usecols=['binds']) 
# y_train = train['binds'].tolist()

# del train
# gc.collect()

# # Split the data into train, validation and test sets
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=RANDOM_STATE)

In [ ]:
# # Create and train the ensemble models
# catboost_model = CatBoostClassifier(random_state=RANDOM_STATE)
# #lgbm_model = LGBMClassifier(random_state=RANDOM_STATE)
# #xgb_model = XGBClassifier(random_state=RANDOM_STATE)

# catboost_model.fit(X_train, y_train)

# map_score = average_precision_score(y_val, catboost_model.predict(X_val))
# print(f"Mean Average Precision (mAP): {map_score:.8f}")

In [ ]:
# del X_train, X_val, y_train, y_val, map_score

In [ ]:
# # Save the Modle to file in the current working directory
# Pkl_Filename = "Model.pkl"  
# with open(Pkl_Filename, 'wb') as file:  
#     pickle.dump(catboost_model, file)

**Chemical Language Model Embeddings**

Just as traditional language models use self-attention mechanisms to compute the representation of each language element (e.g. word in a sentence) to every other element, chemical language models use the same principle in which elements are some chemical unit (e.g. atoms) instead of words.

Below is example code to obtain learned transformer-based embeddings from two chemical language models:

    ChemBERTa: adapted for chemical SMILES from the RoBERTa architecture, trained on a dataset of 77 million molecules
    MoLFormer: another transformer-based model adapted for SMILES but trained on a larger dataset (1.1 billion molecules!)


In [ ]:
from tqdm import tqdm
import torch
from transformers import AutoModel, AutoTokenizer
# load pre-trained ChemBERTa model checkpoint and tokenizer
cb_tokenizer = AutoTokenizer.from_pretrained('DeepChem/ChemBERTa-10M-MLM')
cb_model = AutoModel.from_pretrained('DeepChem/ChemBERTa-10M-MLM')
cb_model.eval()

In [ ]:
# train = pd.read_csv('/kaggle/temp/train.csv', usecols=['molecule_smiles']) 
# train.head()

In [ ]:
num=10000
for i in range(num,extract+1,num):  
    selected_rows = pd.read_csv('/kaggle/temp/train.csv', usecols=['molecule_smiles']).iloc[i-num:i] 
    cb_encoded_inputs = cb_tokenizer(list(selected_rows['molecule_smiles']), padding=True, truncation=True, return_tensors="pt")
    # calculate embeddings
    del selected_rows
    gc.collect()
    
    with torch.no_grad():
        outputs = cb_model(**cb_encoded_inputs)
        
    # extract pooled output
    pd.DataFrame(outputs.pooler_output.numpy().astype(np.float16)).to_csv('/kaggle/temp/cb_embeddings.csv', index=False, mode='a',header=not os.path.exists('/kaggle/temp/cb_embeddings.csv'))
    del cb_encoded_inputs, outputs 
    gc.collect()
    

In [ ]:
cb_embeddings_df = pd.read_csv('/kaggle/temp/cb_embeddings.csv',dtype='float16')
cb_embeddings_df.info()

In [ ]:
# cb_embeddings_df.head()

In [ ]:
train_series = pd.Series()
for x in range(cb_embeddings_df.shape[0]):
    train_series[x] = cb_embeddings_df.iloc[x].squeeze().tolist()
    
del cb_embeddings_df
gc.collect()

In [ ]:
# One-hot encode the protein_name
train = pd.read_csv('/kaggle/temp/train.csv', usecols=['protein_name']) 

onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(train['protein_name'].values.reshape(-1, 1))
del train 
gc.collect()

In [ ]:
# Combine ECFPs and one-hot encoded protein_name
X_train = [ecfp + protein for ecfp, protein in zip(train_series.tolist(), protein_onehot.tolist())]

del train_series,protein_onehot
gc.collect()


train = pd.read_csv('/kaggle/temp/train.csv', usecols=['binds']) 
y_train = train['binds'].tolist()

del train
gc.collect()


# Split the data into train, validation and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=RANDOM_STATE)

In [ ]:
# Create and train the ensemble models
catboost_model = CatBoostClassifier(random_state=RANDOM_STATE)
#lgbm_model = LGBMClassifier(random_state=RANDOM_STATE)
#xgb_model = XGBClassifier(random_state=RANDOM_STATE)

catboost_model.fit(X_train, y_train)

map_score = average_precision_score(y_val, catboost_model.predict(X_val))
print(f"Mean Average Precision (mAP): {map_score:.8f}")

In [ ]:
del X_train, X_val, y_train, y_val, map_score
gc.collect()

**PART 2: testing**

In [ ]:
if os.path.exists("/kaggle/working/submission.csv"):
  os.remove("/kaggle/working/submission.csv")

In [ ]:
# # Load the Model back from file
# Pkl_Filename = "/kaggle/working/Model.pkl"  
# with open(Pkl_Filename, 'rb') as file:  
#     Pickled_Model = pickle.load(file)

# Pickled_Model.predict(X_test)

In [ ]:
output_file = '/kaggle/working/submission.csv'   
test_path = '/kaggle/input/leash-BELKA/test.parquet'
num = 5000
offset_value = 0
#submission should have 1674896 rows
for limit_value in range(num,1675000+1,num):  
    
    con = duckdb.connect()
    test = con.query(f"""(SELECT id, molecule_smiles, protein_name
                        FROM parquet_scan('{test_path}')
                        ORDER BY id
                        LIMIT {num}
                        OFFSET {offset_value})""").df()
    con.close()
    del con
    gc.collect()
    
    cb_encoded_inputs = cb_tokenizer(list(test['molecule_smiles']), padding=True, truncation=True, return_tensors="pt")
    # calculate embeddings
    test=test.drop(columns=['molecule_smiles'])
    gc.collect()
    
    with torch.no_grad():
        outputs = cb_model(**cb_encoded_inputs)
        
    test_protein_onehot = onehot_encoder.transform(test['protein_name'].values.reshape(-1, 1))
    test=test.drop(columns=['protein_name'])  
    gc.collect()
    
    X_test = [ecfp + protein for ecfp, protein in zip(outputs.pooler_output.numpy().astype(np.float16).tolist(), test_protein_onehot.tolist())]
    del cb_encoded_inputs, outputs, test_protein_onehot 
    gc.collect()

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': test['id'], 'binds': catboost_model.predict(X_test)})

    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
    #output_df.head()
    del X_test, test, output_df
    gc.collect()
    
    offset_value=limit_value
    print(limit_value,offset_value)
    